# Test 1

### Packages

In [1]:
import pandas as pd
from datetime import timedelta  
import os

os.chdir('/Users/tristanmayer/Documents/GitHub/taxi-time')
from src.preprocessing import *
from src.master import *

### Data

In [2]:
df1 = cleaning_airport_df('/Users/tristanmayer/Desktop/Supercase Eleven/data/0. Airport data/Airport_Data.csv')

/Users/tristanmayer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [3]:
df2 = get_df_of_obs1(df1)

In [4]:
df3 = get_target_values(df2)

In [5]:
## Creation des timestamps
## Cleaning of rows that font chier la bite
df1 = df1.loc[df1['aldt']!='aldt']
for col in ['aldt', 'aibt','aobt', 'atot']:
    df1[col] = pd.to_datetime(df1[col])

In [6]:
# number of planes in the airport at time_t (a date in string format)
# number of planes in block at time_t (a date in string format)
#### need to reconcile flights with plane (we only know details about flight so not the same line for arrival and departure from the stand -> see the dates regarding one stand)

# number of planes in movement at time_t (a date in string format)
## input: date as a string
## output: integer
def get_nb_of_planes_in_movement(time_t):
    t = pd.to_datetime(time_t)
    res = len(df1[((df1['aldt']<t) & (df1['aibt']>t))|
                  ((df1['aobt']<t) & (df1['atot']>t))])
    return res

# number of planes that have landed on the runway_R in the last M_min at time_t
## input: runway as a string; date as a string; timedelta as an integer (nb of minutes)
## output: integer
def get_nb_of_planes_runway_in_last_M_min(runway_R, time_t, M_min):
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    res = len(df1[(df1['runway']==runway_R) &
                  (df1['aldt']<t1) &
                  (df1['aldt']>t2)])
    return res


# number of planes that have stayed at the stand_S in the last M min at time_t
def get_nb_of_planes_stand_in_last_M_min(stand_S, time_t, M_min):
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    res = len(df1[(df1['stand']==stand_S) &
                  (df1['aibt']<t1) &
                  (df1['aibt']>t2)])
    return res

# average taxi-time of the planes that have landed on runway_R in the last M min at time_t
def average_taxitime_runway_last_X_min(runway_R, time_t, M_min): 
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    list_of_index = df1[(df1['runway']==runway_R) & 
                        (df1['aibt']<t1) & 
                        (df1['aibt']>t2)].index
    res = df3.loc[list_of_index][0].mean()
    return res

# average taxi-time of the planes that stayed at stand_S in the last M min at time_t
def average_taxitime_stand_last_X_min(stand_S, time_t, M_min): 
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    list_of_index = df1[(df1['stand']==stand_S) & 
                        (df1['aibt']<t1) & 
                        (df1['aibt']>t2)].index
    res = df3.loc[list_of_index][0].mean()
    return res

# taxi_time of the last plane that have landed on runway_R at time_t
def get_last_taxitime_runway(runway_R, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.iloc[df1[(df1['runway']==runway_R) & (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

# taxi_time of the last plane that have stayed at stand_S at time_t
def get_last_taxitime_stand(stand_S, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.iloc[df1[(df1['stand']==stand_S) & (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

    
# taxi_time of the last plane that have gone from runway_R to stand_S at time_t
def get_last_taxitime_path(runway_R, stand_S, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.iloc[df1[(df1['runway']==runway_R) &
                       (df1['stand']==stand_S) & 
                       (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

In [ ]:
time_t = '2018-08-01 19:59:00'

In [ ]:
df1.head()

In [ ]:
t = pd.to_datetime(time_t)
len(df1[((df1['aldt']<t) & (df1['aibt']>t))|
        ((df1['aobt']<t) & (df1['atot']>t))])

In [ ]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 300)
len(df1[(df1['runway']=='RUNWAY01')&
        (df1['aldt']<t1) &
        (df1['aldt']>t2)])

In [ ]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 300)
res = len(df1[(df1['stand']=='STAND03') &
                  (df1['aibt']<t1) &
                  (df1['aibt']>t2)])
res

In [ ]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 30000)
list_of_index = df1[(df1['stand']=='STAND03') & 
                    (df1['aibt']<t1) & 
                    (df1['aibt']>t2)].index
res = df3.loc[list_of_index][0].mean()
res

In [ ]:
t1 = pd.to_datetime(time_t)
df3.iloc[df1[(df1['runway']=='RUNWAY01') & (df1['aibt']<t1)]['aibt'].idxmax()][0]
         
         
#.sort_values('aibt', ascending=False).iloc[0].name][0]

In [ ]:
df1[(df1['stand']=='STAND03')]

In [16]:
df1[(df1['runway']=='RUNWAY02') & (df1['aibt']<pd.to_datetime('2018-10-12 13:18:00'))]['aibt'].idxmax()

141628

In [17]:
df3.loc[141628
       ]

,0
4,240.0
7,240.0
8,180.0
9,240.0
11,300.0
14,600.0
15,360.0
16,420.0
17,420.0
18,360.0


## Test des fonctions

In [7]:
get_nb_of_planes_in_movement('2018-10-12 13:18:00')

19

In [8]:
get_nb_of_planes_runway_in_last_M_min('RUNWAY02','2018-10-12 13:18:00', 2000)

8

In [9]:
get_nb_of_planes_stand_in_last_M_min('STAND104', '2018-10-12 13:18:00', 2000)

4

In [10]:
average_taxitime_runway_last_X_min('RUNWAY02','2018-10-12 13:18:00', 2000)

375.0

In [11]:
average_taxitime_stand_last_X_min('STAND104', '2018-10-12 13:18:00', 2000)

510.0

In [12]:
get_last_taxitime_runway('RUNWAY02','2018-10-12 13:18:00')

IndexError: single positional indexer is out-of-bounds

In [13]:
get_last_taxitime_stand('STAND104','2018-10-12 13:18:00')

IndexError: single positional indexer is out-of-bounds

In [14]:
get_last_taxitime_path('RUNWAY02', 'STAND104', '2018-10-12 13:18:00')

ValueError: attempt to get argmax of an empty sequence